In [1]:
import pandas as pd

In [2]:
year = 2012
country = "France"
geography = "department_id"

In [6]:
output = []
for method in ["std", "std_rank", "skew", "kurtosis"]:
    path = f"data_output/{country}/{year}_divisiveness_{geography}_{method}.csv.gz"
    tmp = pd.read_csv(path, compression="gzip")
    tmp["method"] = method
    output.append(tmp)
    
df = pd.concat(output, ignore_index=True)

In [17]:
df[df["candidate"] == "Marine LE PEN"].pivot(index="department_id", columns="method", values="value").corr("kendall")

method,kurtosis,skew,std,std_rank
method,,,,
kurtosis,1.000000,0.391574,0.000396,-0.095842
skew,0.391574,1.000000,-0.201188,-0.089901
std,0.000396,-0.201188,1.000000,0.282314
std_rank,-0.095842,-0.089901,0.282314,1.000000
